### Step 1 - Create Training Data

In [ ]:
import cv2
import numpy as np

In [ ]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

In [ ]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

In [ ]:
# Collect 100 samples of your face from webcam input
# C:\Users\mukes\Pictures\faces
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = 'C:/Users/mukes/Pictures/faces/user1/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples1 Complete")

### Step 2 - Train Model

In [ ]:
cap.release()

In [ ]:
from os import listdir
from os.path import isfile, join

In [ ]:
count = 1
face_model=[]
while True:
    # Get the training data we previously made
    data_path = 'C:/Users/mukes/Pictures/faces/user'+str(count)+'/'
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

    # Create arrays for training data and labels
    Training_Data, Labels = [], []

    # Open training images in our datapath
    # Create a numpy array for training data
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)

    # Create a numpy array for both training data and labels
    Labels = np.asarray(Labels, dtype=np.int32)

    # Initialize facial recognizer
    # model = cv2.face.createLBPHFaceRecognizer()
    # NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
    # pip install opencv-contrib-python
    # model = cv2.createLBPHFaceRecognizer()

    face_model.append(cv2.face_LBPHFaceRecognizer.create()) 
    # Let's train our model 
    face_model[count-1].train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model"+str(count)+" trained sucessefully")
    count = count +1
    if(count > 5):
        break

### Step 3 - Action To be Performed

In [ ]:
team_data = [
    ["Mukesh","+91_mobile_no","mail@gmail.com"],
    ["Lovely","+91_mobile_no", "mail@gmail.com"],
    ["Swati","+91_mobile_no", "mail@gmail.com"],
    ["Harshita","+91_mobile_no", "mail@gmail.com"],
    ["Sunil","+91_mobile_no", "mail@gmail.com"],
]


In [ ]:
#Mail
import smtplib

def mail(i):
    try:
        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.starttls()
        s.login("sender_username", "password")
        message = "This is face of "+ team_data[i][0]
        s.sendmail("sender_email id",team_data[i][2], message)
        print("Mail Sent to, "+ team_data[i][0])
        s.quit()
    except:
      print("An Unexpected Error!")

In [ ]:
# WhatsApp
import pywhatkit
import datetime

def whatsapp(i):
    try:
        now = datetime.datetime.now()
        hr = int(now.strftime("%H"))
        min = int(now.strftime("%M")) + 1
 
        pywhatkit.sendwhatmsg(team_data[i][1], "Hello from WhatsApp Automation",hr,min)
        print("Msg Sent to"+ team_data[i][0])

    except:
        print("An Unexpected Error!")

In [ ]:
# For mail you need to enable this 
#https://myaccount.google.com/lesssecureapps?pli=1&rapt=AEjHL4M5NQx9ygzKY4j97NvrKYF1XjgprSQgDgioZ-E090hTDo5BLEm2bEdl9UNmgY6LAkpfX8cisp4BOSvrFRKvIFBYujb6Zw

In [ ]:
import subprocess as sp
import time

def action_two():
    instance_id = sp.getoutput("aws ec2 run-instances --image-id <ami-id> --instance-type t2.micro --count 1 --key-name tryaws --subnet-id <subnet-id>  --security-group-ids <sg-id>  --tag-specifications=ResourceType=instance,Tags=[{Key=Name,Value=CV_instances}]  --query Instances[*].[InstanceId] --output text")
    print("Instance Created with id:")
    print(instance_id)
    vol_id = sp.getoutput("aws ec2 create-volume --availability-zone ap-south-1a --size 5 --tag-specifications ResourceType=volume,Tags=[{Key=Name,Value=CV_volume}] --query VolumeId --output text")
    print("Volume Created with id:")
    print(vol_id) 
    time.sleep(10)
    sp.getoutput("aws ec2 attach-volume --volume-id {} --instance-id {} --dev /dev/sdf".format(vol_id,instance_id))
    print("Volume Attached to Instance")

### Step 4 - Run Our Facial Recognition

In [ ]:
import sys

In [ ]:
def menu(i):
    print('''
        ================Menu====================
        1. Whatsapp
        2. Email
        3. AWS
        ========================================
    ''')
    inp = int(input("Enter Your Choice: "))
    if(inp==1):
        whatsapp(i)
        sys.exit()
    if(inp==2):
        mail(i)
        sys.exit()
    if(inp==3):
        action_two()
        sys.exit()
    else:
        print("Wrong Input!! Try Again....")
        sys.exit()
        

In [ ]:
# Check Confidence and Display data...
def get_confidence(results):
    confidence=[]
    for i in range(0,5):
        if results[i][1] < 500:
            confidence.append(int( 100 * (1 - (results[i][1])/400) )) 
    
    for i in range(0,5):
        if confidence[i]>88:
            display_string = str(confidence[i]) + '% Confident it is User'
            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.putText(image, "Hey "+team_data[i][0], (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            menu(i)

    else:
        cv2.imshow('Face Recognition', image )
            

In [ ]:
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results=[]
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        for i in range(0,5):
            results.append(face_model[i].predict(face))
            
        #Get_Confidence
        get_confidence(results)

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()       